In [1]:
import pandas as pd
import os
import sys
from datetime import datetime
import csv
import concurrent.futures
import subprocess
from datetime import datetime

Dataset of work "SZZ in the time of Pull Requests" 
- SOURCE: https://github.com/mozilla/regressors-regressions-dataset

In [2]:
# Clone Repository
# !hg clone https://hg.mozilla.org/mozilla-central/

In [32]:
# Download dataset
# !wget https://raw.githubusercontent.com/mozilla/regressors-regressions-dataset/main/dataset.csv

In [3]:
def getRevisionId(commit_hash):
    return subprocess.getoutput("cd firefox-source && hg id -n -r "+commit_hash)

In [4]:
def calculateDistanceCommits(bfc,bic):
    aux= bic + "::" + bfc
    distance = subprocess.getoutput("cd firefox-source && hg log --rev "+aux+" --template \"{rev} \" | wc -w")
    return int(distance)

In [5]:
def getDate(c_id):
    date = subprocess.getoutput("cd firefox-source && hg log -r "+c_id+" --template \"{date|date}\n\"")
    return datetime.strptime(date, "%a %b %d %H:%M:%S %Y %z")
def calculateDistanceDays(bfc_id,bic_id):
    bfc_date = getDate(bfc_id)
    bic_date = getDate(bic_id)
    return (bfc_date-bic_date).days

In [6]:
def addDistances(link):
    bfc = link['FIX_COMMITS_MERCURIAL'].split(" ")[0]
    bfc_id = getRevisionId(bfc)
    if isinstance(link['BUG_COMMITS_MERCURIAL'], str):
        bic = link['BUG_COMMITS_MERCURIAL'].split(" ")[0]
        bic_id = getRevisionId(bic)
    else:
        link['CALCULABLE_DISTANCE'] = False
        return
    link['DISTANCE_COMMITS'] = calculateDistanceCommits(bfc, bic)
    link['DISTANCE_DAYS'] = calculateDistanceDays(bfc_id,bic_id)
    link['CALCULABLE_DISTANCE'] = True

In [26]:
df = pd.read_csv('dataset.csv')
links = df[df['UNLINKABLE']==False].to_dict('records')
#links = df.to_dict('records')
len(links)

4174

In [27]:
future_results = []
with concurrent.futures.ThreadPoolExecutor(64) as executor:
    for link in links:
        future = executor.submit(addDistances, link)
        future_results.append(future)
for future in future_results:
    try:
        future.result() 
    except Exception as e:
        print(e)

In [28]:
df = pd.DataFrame.from_records(links)
df[['DISTANCE_COMMITS','DISTANCE_DAYS']].describe().T

,count,mean,std,min,25%,50%,75%,max
DISTANCE_COMMITS,4174.0,13881.897221,37878.281626,0.0,714.0,2604.5,10937.75,514394.0
DISTANCE_DAYS,4174.0,92.101342,269.759532,-933.0,5.0,18.0,72.00,4668.0


In [33]:
# There are 70 commits (from 4174) commits where BFC was created BEFORE BIC (???)
df[df['DISTANCE_DAYS'] >= 0]

,FIX_ID,FIX_COMMITS_MERCURIAL,FIX_COMMITS_GIT,BUG_ID,BUG_COMMITS_MERCURIAL,BUG_COMMITS_GIT,NO_FILE_SHARED_FILES,NEW_LINES_ONLY_FIX,REMOVE_LINES_ONLY_BUG,NO_BUG,UNLINKABLE,DISTANCE_COMMITS,DISTANCE_DAYS,CALCULABLE_DISTANCE
0,724457,31aea004045a5572adb1a665fca15f19531b4a42,830f666746d9096b664f9cd7dc00b182011bd57d,610902,e35dfafeee4222dd02fb04c192dcd10eb24cc129,b44beb2f50adea62fb1d4c44b88c1231cd93efae,False,False,False,False,False,378769,2690,True
1,992384,98575a5ca794cb67b8d569e8a95188a30532e336,a3329bf3430074eaf2d41dc1b74c95019bb59d6b,1102175,004a0cb8af9fbbe84522d0506a1fdd132d51944c c4285...,d5fdf5222cfcf502466759410a076018e3220114 18826...,False,False,False,False,False,1213,8,True
2,1221889,e366c310a5eb2204c77288c2450727aacef5d773,ef2909de413dfb64cd4d68ae2c4958bcc09f0e4a,1477205 1205558,f1f82e673a3490bb572c3a1110eeded81badb9a9 d89b9...,ca25e7dbb6892b3ce6cd364277e6d8fe5fc60c95 89463...,False,False,False,False,False,240161,1597,True
3,1224848,606676e2dd214e37cee9bae611072918cbd87df7,eec931567c3e973d10701d0c6b090a4e29a0de3c,786674,f5c7c5cd504a912dd8bd1e5775c5accb97fc7a32,d11f3d18f453b646ae5619440d0a83cee726beae,False,False,False,False,False,475,2,True
4,1242221,648bcac47c6c96531732742440280d25fbd80279,8a0a23fcb6eba6e5e10d18fc01083c9330f5ddf5,1563821,6aebc76dace8f9a68bc229bfefcdb83af1fd07fb,01ff3d19c2f9ceaf9e8511b0b3a39eb64967b7e5,False,False,False,False,False,2151,13,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4169,1676414,2d93ec1fdb2517df6c92bf83ab66d0ca303da668,ea186e67796833032f0ed8dffe61e1171cd49783,1675549,ab2514a6da6491cffeca1445b1914826975d3395,39ed8503d344b2628e07f750fc1fef1e6cbc0fa4,False,False,False,False,False,147,0,True
4170,1676417,d3281939949e31fc36141751fc90206701a6c78f,37f3e27c37ad95222e17095e73713d6d7e36cd9a,1674515,5d8fba30a3f4b2550e26bc55029bb7a66dbd7484 ca520...,beeb9bf6e2990d82f01ae198f1c83bc3a2187164 bd2dd...,False,False,False,False,False,119,0,True
4171,1617111,2bb5b1e1caf647c617838c455527a4b7fa25431b,2de30cc56350c503a7b0b1f427a8ebb7bffd5005,1615607,95fe1470b3718d04e5c45605faedda62da140071,a019e5acf2b9584e512f47ff3c5d3369e6cabe4d,False,False,False,False,False,4921,28,True
4172,1623938,7172c878c66e6b997d823ce37c6f9491e856bbae,704a04f8bf2f119b40a0881d8273309eb3b149e1,1606036,dbeea4db8edeac1d346210eccbd1ed0240d5f65f,0bfa1c9de00e2d534a9f97f3427ced361c1c3702,False,False,False,False,False,11745,82,True


In [23]:
df[df['DISTANCE_DAYS'] >= 0 & df['CALCULABLE_DISTANCE']][['DISTANCE_COMMITS','DISTANCE_DAYS']].describe().T

,count,mean,std,min,25%,50%,75%,max
DISTANCE_COMMITS,4104.0,14118.278265,38156.376317,0.0,769.0,2742.5,11158.75,514394.0
DISTANCE_DAYS,4104.0,95.172027,270.068372,0.0,5.0,19.0,75.00,4668.0
